# To compare between models

In [ ]:
# TO_COMPARE = ["2022_03_27-20_51_13", "2022_03_27-20_51_29", "2022_03_27-20_51_43"] # lr from scratch
# TO_COMPARE = ["2022_03_28-02_01_58","2022_03_28-02_02_28","2022_03_28-02_02_38","2022_03_28-02_02_43","2022_03_28-18_15_16"] # lr
# TO_COMPARE = ["2022_03_28-18_15_38","2022_03_28-18_23_16","2022_03_29-04_12_27","2022_03_29-04_14_51"] # bs
# TO_COMPARE = ["2022_03_28-02_02_38", "2022_03_29-21_05_24", "2022_03_29-21_05_31"] # crop
TO_COMPARE = ["2022_03_31-18_37_59_gpK","2022_03_31-18_38_39_AqQ","2022_03_31-18_56_57_xfY","2022_03_28-02_02_38","2022_04_01-04_25_45_DEW","2022_04_01-18_56_08_GUY"]

COMPARE_ON_EPOCH = 40

NB_IMAGES = 20

BATCH_SIZE = 5

WIDTH = 8
HEIGHT = 8

In [ ]:
def get_name(row):
    epoch_nb = min(COMPARE_ON_EPOCH, row["epochs_trained_nb"])
    # return f"{row['run_id']} on {epoch_nb} with lr {row['training_gen_lr']}" # lr
    # return f"{row['run_id']} on {epoch_nb} with batch size {row['training_batch_size']}" # bs
    # return f"{row['run_id']} on {epoch_nb} with crop mode {row['cartoon_dataset_crop_mode']}" # crop
    return f"{row['run_id']} on {epoch_nb} with content loss {row['training_weight_generator_content_loss']}" # content loss

In [ ]:
%cd ../..

In [ ]:
import os
import pandas as pd
from ast import literal_eval
from math import ceil

from scripts import config
from scripts.tools.get_name import get_gen_name, get_model_img_name
from scripts.tools.type_dict.csv_to_object import architecture_dict, crop_mode_dict, ratio_filter_dict
from src.pipelines.cartoonizer import Cartoonizer
from src import models, dataset
import matplotlib.pyplot as plt

In [ ]:
def get_cartoonizer(df_runs, run_id):
    run = df_runs.loc[df_runs['run_id'] == run_id].iloc[0]
    epoch_nb = min(COMPARE_ON_EPOCH, run["epochs_trained_nb"])
    gen_path = os.path.join(
        config.WEIGHTS_FOLDER,
        run["run_id"],
        get_gen_name(epoch_nb),
    )

    pictures_dataset_parameters = dataset.PicturesDatasetParameters(
        new_size=literal_eval(run["picture_dataset_new_size"]),
        crop_mode=crop_mode_dict[run["picture_dataset_crop_mode"]],
        ratio_filter_mode=ratio_filter_dict[run["picture_dataset_ratio_filter_mode"]],
        nb_images=NB_IMAGES,
    )

    cartoonizer = Cartoonizer(
        infering_parameters=models.InferingParams(batch_size=BATCH_SIZE),
        architecture=architecture_dict[run["cartoon_gan_architecture"]],
        architecture_params=models.ArchitectureParamsNULL(),
        pictures_dataset_parameters=pictures_dataset_parameters,
        gen_path=gen_path,
    )

    return get_name(run), cartoonizer

In [ ]:
def get_path(df_runs, run_id):
    run = df_runs.loc[df_runs['run_id'] == run_id].iloc[0]
    epoch_nb = min(COMPARE_ON_EPOCH, run["epochs_trained_nb"])
    pictures_path = os.path.join(config.LOGS_FOLDER, run_id, "pictures")
    epoch_pictures_path = os.path.join(
        pictures_path, f"epoch_{epoch_nb}"
    )
    return {"name": get_name(run), "path": epoch_pictures_path}

In [ ]:
df_runs = pd.read_csv(config.REMOTE_PARAMS_PATH, index_col=0)
all_paths = {run_id: get_path(df_runs, run_id) for run_id in TO_COMPARE}

for i in range(NB_IMAGES):
    nb_models = len(TO_COMPARE)
    cols = 2
    rows = ceil((nb_models+1)/cols)
    fig = plt.figure(figsize=(WIDTH*cols, HEIGHT*rows))
    picture = plt.imread(os.path.join(all_paths[TO_COMPARE[0]]['path'], get_model_img_name(i, "picture")))
    ax = fig.add_subplot(rows, cols, 1)
    ax.axis("off")
    ax.imshow(picture)
    plt.title("Original picture")
    for j in range(len(TO_COMPARE)):
        run_id = TO_COMPARE[j]
        cartoon = plt.imread(os.path.join(all_paths[run_id]['path'], get_model_img_name(i, "cartoon")))
        ax = fig.add_subplot(rows, cols, j+2)
        ax.imshow(cartoon)
        plt.title(all_paths[run_id]['name'])
        ax.axis("off")